<a href="https://colab.research.google.com/github/danielcbeaulieu/NLP_portfolio/blob/master/toxic_multilabel_intel_07282019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls '/content/drive/My Drive/uspto/models'

In [0]:
!cd '/content/drive/My Drive/uspto'

In [4]:
!google-drive-ocamlfuse uspto

/bin/bash: google-drive-ocamlfuse: command not found


In [5]:
!pip uninstall apex

Uninstalling apex-0.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/apex-0.1-py3.6-linux-x86_64.egg
Proceed (y/n)? y
  Successfully uninstalled apex-0.1


In [6]:
!pip install feather-format

In [7]:
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!python setup.py install --cuda_ext --cpp_ext

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
torch.__version__  =  1.1.0

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
creating build/bdist.linux-x86_64/egg
copying build/lib.linux-x86_64-3.6/fused_layer_norm_cuda.cpython-36m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg
copying build/lib.linux-x86_64-3.6/syncbn.cpython-36m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86

In [8]:
%cd ..

/content


In [9]:
!ls

apex  california_cities.csv  drive  sample_data


In [10]:
!pip install pytorch_pretrained_bert
!pip install pathlib
!pip install fastai
!pip install pandas
!pip install collections
!pip install os
!pip install tqdm
!pip install sys
!pip install random
!pip install numpy
!pip install sklearn
!pip install datetime
!pip install fast-bert
!pip install feather-format

  ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
  ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
  ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys
  ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [0]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertConfig, BertForMaskedLM, BertForSequenceClassification
from pathlib import Path
import torch

from fastai.text import Tokenizer, Vocab
import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split

import datetime
    
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pytorch_pretrained_bert.optimization import BertAdam

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [0]:
torch.cuda.empty_cache()

In [0]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

In [0]:
!ls '/content/drive/My Drive/uspto/models/'

In [0]:
DATA_PATH = Path('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data/')
LABEL_PATH = Path('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/label/')

AUG_DATA_PATH = Path('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data_aug/')

MODEL_PATH=Path('/content/drive/My Drive/uspto/models/')
LOG_PATH=Path('/content/drive/My Drive/uspto/logs/')
MODEL_PATH.mkdir(exist_ok=True)

model_state_dict = None

#BERT_PRETRAINED_PATH = Path('/content/drive/My Drive/uspto/pretrained_models/fast-bert/cased_L-12_H-768_A-12/')
BERT_PRETRAINED_PATH = Path('/content/drive/My Drive/uspto/pretrained_models/fast-bert/bert-base-uncased/bert_config.json')
#BERT_PRETRAINED_PATH = Path('/drive/My Drive/uspto/pretrained_models/fast-bert/uncased_L-12_H-768_A-12/')
#BERT_PRETRAINED_PATH = Path('/drive/My Drive/uspto/pretrained_models/fast-bert//uncased_L-24_H-1024_A-16/')
#FINETUNED_PATH = Path('/drive/uspto/My Drive/pretrained_models/fast-bert/finetuned_model.bin')
FINETUNED_PATH = None
# model_state_dict = torch.load(FINETUNED_PATH)

LOG_PATH.mkdir(exist_ok=True)

In [16]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59658  100 59658    0     0   217k      0 --:--:-- --:--:-- --:--:--  217k


In [0]:
args = {
    "run_text": "multilabel toxic comments with freezable layers",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "toxic_classification_lib",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": MODEL_PATH/'output',
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "learning_rate": 5e-6,
    "num_train_epochs": 4.0,
    "warmup_proportion": 0.1,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "loss_scale": 128
}

In [0]:
import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()













In [19]:
logger.info(args)

07/30/2019 00:18:08 - INFO - root -   {'run_text': 'multilabel toxic comments with freezable layers', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('/content/drive/My Drive/uspto/logs'), 'full_data_dir': PosixPath('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data'), 'data_dir': PosixPath('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data'), 'task_name': 'toxic_classification_lib', 'no_cuda': False, 'bert_model': PosixPath('/content/drive/My Drive/uspto/pretrained_models/fast-bert/bert-base-uncased/bert_config.json'), 'output_dir': PosixPath('/content/drive/My Drive/uspto/models/output'), 'max_seq_length': 512, 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 16, 'eval_batch_size': 16, 'learning_rate': 5e-06, 'num_train_epochs': 4.0, 'warmup_proportion': 0.1, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': True, 'loss_scale': 128}


In [20]:
#tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=args['do_lower_case'])
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
 

07/30/2019 00:18:08 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
device = torch.device('cuda')
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [0]:
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [23]:
!ls '/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data/'

test.csv  train.csv  train_sample.csv  val_sample.csv


In [24]:
!ls '/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data/'

test.csv  train.csv  train_sample.csv  val_sample.csv


In [0]:
#train = pd.read_csv('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data/train.csv')
#test = pd.read_csv('/content/drive/My Drive/uspto/data/multi_label_toxic_comments/data/test.csv')

In [26]:
#print(test.head())

NameError: ignored

In [0]:
databunch = BertDataBunch(args['data_dir'], LABEL_PATH, tokenizer, train_file='train_sample.csv', val_file='val_sample.csv',
                          test_data='test.csv',
                          text_col="comment_text", label_col=label_cols,
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)
databunch.save()

In [0]:
# databunch = BertDataBunch.load(args['data_dir'])

In [0]:
num_labels = len(databunch.labels)

In [0]:
# torch.distributed.init_process_group(backend="nccl", 
#                                      init_method = "tcp://localhost:23459", 
#                                      rank=0, world_size=1)

In [0]:
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})
metrics.append({'name': 'accuracy_single', 'function': accuracy_multilabel})

In [37]:
databunch

In [38]:
learner = BertLearner.from_pretrained_model(databunch, 'bert-base-uncased', metrics, device, logger, 
                                            finetuned_wgts_path=FINETUNED_PATH, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)

07/30/2019 00:37:46 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmp22asyd30


100%|██████████| 407873900/407873900 [00:06<00:00, 66623026.10B/s]

07/30/2019 00:37:53 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmp22asyd30 to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba


07/30/2019 00:37:53 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
07/30/2019 00:37:53 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmp22asyd30
07/30/2019 00:37:53 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
07/30/2019 00:37:53 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpv_mm6z8u
07/30/2019 00:37:57 -

In [39]:
learner.fit(4, lr=args['learning_rate'], schedule_type="warmup_cosine_hard_restarts")

07/30/2019 00:43:43 - INFO - root -   Loss after epoch 0 - 0.4255797371031746
07/30/2019 00:43:43 - INFO - root -   Running evaluation


07/30/2019 00:44:18 - INFO - root -   Eval results:
07/30/2019 00:44:18 - INFO - root -     eval_loss = 0.16981627449156747
07/30/2019 00:44:18 - INFO - root -     metrics = {'accuracy_thresh': 0.9703333377838135, 'roc_auc': 0.8212557752654597, 'fbeta': 0.0, 'accuracy_single': 0.885}
07/30/2019 00:44:18 - INFO - root -   --------------------------------------------------------------------------------
07/30/2019 00:45:00 - INFO - root -   Loss after epoch 1 - 0.14656963045634921
07/30/2019 00:45:00 - INFO - root -   Running evaluation


07/30/2019 00:45:34 - INFO - root -   Eval results:
07/30/2019 00:45:34 - INFO - root -     eval_loss = 0.11136397104414683
07/30/2019 00:45:34 - INFO - root -     metrics = {'accuracy_thresh': 0.9703333377838135, 'roc_auc': 0.9447350036089379, 'fbeta': 0.014424622058868408, 'accuracy_single': 0.884}
07/30/2019 00:45:34 - INFO - root -   --------------------------------------------------------------------------------
07/30/2019 00:46:17 - INFO - root -   Loss after epoch 2 - 0.10399664015997023
07/30/2019 00:46:17 - INFO - root -   Running evaluation


07/30/2019 00:46:50 - INFO - root -   Eval results:
07/30/2019 00:46:50 - INFO - root -     eval_loss = 0.09823123992435516
07/30/2019 00:46:50 - INFO - root -     metrics = {'accuracy_thresh': 0.9749999642372131, 'roc_auc': 0.9497194871062493, 'fbeta': 0.034099411219358444, 'accuracy_single': 0.969}
07/30/2019 00:46:50 - INFO - root -   --------------------------------------------------------------------------------
07/30/2019 00:47:33 - INFO - root -   Loss after epoch 3 - 0.08400956411210317
07/30/2019 00:47:33 - INFO - root -   Running evaluation


07/30/2019 00:48:06 - INFO - root -   Eval results:
07/30/2019 00:48:06 - INFO - root -     eval_loss = 0.09411088247147817
07/30/2019 00:48:06 - INFO - root -     metrics = {'accuracy_thresh': 0.9749999642372131, 'roc_auc': 0.9489465568417353, 'fbeta': 0.035114556550979614, 'accuracy_single': 0.948}
07/30/2019 00:48:06 - INFO - root -   --------------------------------------------------------------------------------


/bin/bash: line 0: cd: /drive: No such file or directory


In [41]:
!ls -l

total 72
drwxr-xr-x 11 root root  4096 Jul 29 18:13 apex
-rw-r--r--  1 root root 59658 Jul 30 00:18 california_cities.csv
drwx------  3 root root  4096 Jul 29 19:52 drive
drwxr-xr-x  1 root root  4096 Jul 19 16:14 sample_data


In [42]:
%pwd

'/content'

In [54]:
learner.save_and_reload(MODEL_PATH, "trained_toxic_comment")

07/30/2019 01:14:22 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
07/30/2019 01:14:22 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpgk17vc68
07/30/2019 01:14:26 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size